## Filtering
The filtering was done using a Kalman Filter. We designed it to use the position and the angle given by the camera as measurement and the speed of the robot, given by the sensors of the wheels, as prediction. For simplicity of use, we decided to create a Kalman_Filter class, so we would have less arguments to give to each functions. However, this class needs to have access to the class Vision and to be given the speed of the robot to work properly.  


### Equations
The Kalman filter merge together a prediction of the future state of the system with a measure of these states to be more precise. The prediction is govern by the system equations. In this section, we are going to describe the equations used in the model and the ones used to implement the merging part of the filter. 
First, we need to convert the informations we get from the sensors to something we can use:
$v = (v_r + v_l)/2$. This is the translational speed and $w = (v_r - v_l)/b$, which is the rotationnal speed. $v_r$ is the speed of the right wheel, $v_l$ the left one and $b$ is the width of the robot, wheel to wheel. These speed are given in thymio units, so we need to convert them in mm or in pixels to use them with the other values. For w, we just want to change it to mm because b is given in mm and for v, the position are given in pixels unit so we have to convert it first to mm and then to pixels.
From $w$, we can compute the variation of angle that the robot does when it moves: $\alpha = w * \delta t$.
We have three states for our filter, which are $x$, $y$, and $\theta$, respectively the position on the x and y axis and the orientation from the x axis. The state vector is called $\rho$ and the vector of input ($[v ; w]$) is called u for simplication of writing. Therefore, the model equation is the following: $\rho_{k+1} = A * \rho_k + B * u_k$. We now need to determine the matrixes A and B. For this, we have to know how the system evolve with each steps. For x, we can find that $x_{k+1} = x_k + v * \cos(\alpha + \theta) * \delta t$ and for y: $y_{k+1} = y_k + v *\sin(\alpha + \theta)* \delta t$. Finally, for $\theta$, we have $\theta_{k+1} = \theta_k + w *\delta t$. From this we can find that the matrix A is  \begin{bmatrix} 1 & 0 & 0\\ 0 & 1 & 0 \\ 0 & 0 & 1\end{bmatrix} and the B matrix  \begin{bmatrix} \cos(\alpha + \theta) * \delta t & 0\\ \sin(\alpha + \theta) * \delta t & 0 \\ 0 & \delta t \end{bmatrix}
We now have a model for the predictive part of our algorithm.

In [44]:
import numpy as np
pos = [50, 60, np.pi/4]
speed = [200, 200]
b = 100
THYMIO_SPEED_2_MM = 0.435
mm2px = 0.5
time = 0.1
P_est = 1000 * np.ones(3)
Q = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0.1]])

v = mm2px * THYMIO_SPEED_2_MM *(speed[0] + speed[1])/2
w = THYMIO_SPEED_2_MM *(speed[0] - speed[1])/b
alpha = w * time
rho_est = pos
u = np.array([v, w])

A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
B = np.array([[time * np.cos(alpha + rho_est[2]), 0], [time * np.sin(alpha + rho_est[2]), 0], [0, time]])

We can now start the real algorithm of our filter: we start by computing the estimation of our states $\rho_{est\text{ }a\text{ }priori}$ with our model equation. We can then compute the Covariance matrix P: $P_{est \text{ }a\text{ }priori} = A * P_{est} * A^T + Q$ where Q is the error covariance matrix for the estimation and $Q =\begin{bmatrix} 1 & 0 & 0\\ 0 & 1 & 0 \\ 0 & 0 & 0.1\end{bmatrix}$

In [45]:
rho_est_a_priori = np.dot(A, rho_est) + np.dot(B, u)
P_est_a_priori = np.dot(A, np.dot(P_est, A.T)) + Q

now we need to see if the camera gives us measurement or not. This sadly is only possible with the camera connected so for this demo we are setting a variable acting like the function robot_detected() from our vision module. 

#### Camera connected
If the camera is detected, we set the measurement as the values retourned by the camera, $pos_x$, $pos_y$ and $\theta$. This will be our measurement vector $y$. We also compute the covariance matrix for measurement, $S = C * P_{est \text{ }a\text{ }priori} * C^T + R$, where $C$ is the matrix linking measurements and states and $R$ is the error covariance matrix for the measurement. Finally, we compute the Kalman gain, which is the gain for the measurements $K = P_{est \text{ }a\text{ }priori} * C^T * S^{-1}$.
We chose to have a small R compared to Q because we want to put much more weight on the camera measurements, the speed sensors being less precise. That way, when our camera is active the kalman filter will output almost the same position as the position given by the camera. 

In [46]:
C = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
R = np.array([[0.1, 0, 0], [0, 0.1, 0], [0, 0, 0.01]])
robot_detected = 1

#### Camera not connected
If the camera is not connected we set the kalman gain to 0 and therefore we do not compute $S$. For $y$, we simply set it to the last known position.

In [47]:
if robot_detected:   
    y = pos
    S = np.dot(C, np.dot(P_est_a_priori, C.T)) + R
    K = np.dot(P_est_a_priori, np.dot(C.T, np.linalg.inv(S)))
else:
    K = 0
    y = rho_est

Finally, we compute the innovation (the difference between the measured positions and the estimated positions) $i = y - C * \rho_{est\text{ }a\text{ }priori}$. We can now compute the new estimation of the position $\rho_{est} = \rho_{est\text{ }a\text{ }priori} + K * i$ and the new covariance matrix $P_{est} = (I - K * C) * P_{est\text{ }a\text{ }priori}$


In [48]:
i = y - np.dot(C, rho_est_a_priori)
rho_est = rho_est_a_priori + np.dot(K, i)
P_est = np.dot((np.identity(3) - np.dot(K, C)), P_est_a_priori)

To see the effect of the filter, we can print the value of $\rho_{est}$ and the position given by the camera. Of course, with this example being a static one, the position given by the estimate won't be the same as the one seen by the camera, because the camera gives us the last position and the estimate the next position, but it allows us to see the effect of the filter, for example by changing the speed.

In [53]:
print(rho_est, "position estimée")
print(pos, "position given by camera")

[50.2330281  60.2330281   0.73879767] position estimée
[50, 60, 0.7853981633974483] position given by camera
[[1001.  1000.  1000. ]
 [1000.  1001.  1000. ]
 [1000.  1000.  1000.1]]
